<a href="https://colab.research.google.com/github/Photopoint/metadata/blob/main/Photopoint_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Things you need to configure
This script assumes that each photographer will have a main folder on the Google Drive into which they create subfolders and upload their JPEG photos.
The main folder path will be in the root folder ("My Drive") and called "Incoming". Each photographer has one line in the code below in the format:

> `"foldername": "Photographer Name",`

Getting the punctuation correct is vital. The "Photographer Name" part should be their name as they want it to appear on the watermark.

Photographers will need to have write access to "`My Drive/Incoming/foldername`" which for the example of Morgan Churchill would be the folder


> `https://drive.google.com/drive/u/0/my-drive/Incoming/Morgan`

Images will be resized for social media. 1200 pixels (as of Feb 2023) is a good size for Facebook and Instagram limits. If that needs to be different, change the line that defines the `soc_med_size` variable.

In [ ]:
incoming_folder = "/my-drive/Incoming/"
processed_folder = "/my-drive/Processed/"

# the size, in pixels, of the longest edge for social media images
soc_med_size = 1200

# details 
photographers = {
    "Morgan": "Morgan Churchill",
    "Quinn":  "Q Madhani",
}

In [ ]:
import piexif
import PIL
from PIL import Image, ImageDraw, ImageFont
import io
from googleapiclient.discovery import build

def add_watermark(image, font, photographer):
    # Get image width and height
    width, height = image.size

    # Create an ImageDraw object
    draw = ImageDraw.Draw(image)

    # Load the font
    font = ImageFont.truetype(font, 30)

    # Calculate the size of the text
    text_width, text_height = draw.textsize(photographer, font)

    # Calculate the position of the text
    x = width - text_width - 10
    y = height - text_height - 10

    # Add the text to the image
    draw.text((x, y), photographer, font=font, fill=(255, 255, 255, 128))

    return image

def save_image_to_drive(image, file_id, content_type):
    # Save the image with the watermark to a memory buffer
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG")

    # Get the Google Drive API client
    drive_service = build("drive", "v3")

    # Update the existing file with the new image data
    file = drive_service.files().update(fileId=file_id, media_body=buffer.getvalue(),
                                         mimetype=content_type).execute()

def resize_image(image):
    # Calculate the aspect ratio
    aspect_ratio = image.width / image.height

    # Resize the image to be soc_med_size pixels on the long edge
    if aspect_ratio > 1:
        new_width = soc_med_size
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = soc_med_size
        new_width = int(new_height * aspect_ratio)

    return image.resize((new_width, new_height), resample=PIL.Image.LANCZOS)

# Main function
if __name__ == "__main__":
    # Path to the font
    font_path = "arial.ttf"

    # ID of the Google Drive folder
    folder_id = "your-folder-id"

    # Get the Google Drive API client
    drive_service = build("drive", "v3")

    # Query for all JPEG files in the folder
    query = "mimeType='image/jpeg' and trashed = false and parents in '" + folder_id + "'"
    results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()

    # Process each file
    for file in results.get("files", []):
        # Download the image data from Google Drive
        image_data = drive_service.files().get_media(fileId=file["id"]).execute()
        image = Image.open(io.BytesIO(image_data))

        # Get the photographer's name from the EXIF data
        exif_dict = piexif.load(image.info["exif"])
        if piexif.ExifIFD.Artist in exif_dict["0th"]:
            photographer = exif_dict["0th"][piexif.ExifIFD.Artist].decode()
        else:
            photographer = "Unknown"

        # Add the watermark to the image
        watermarked_image = add_watermark(image, font_path, photographer)

        # Get the subfolder "Archive" if it exists, otherwise create one
        archive_folder = get_or_create_subfolder(file["parents"][0], "Archive")

        # Save the original image to the "Archive" folder
        save_image_to_drive(watermarked_image, archive_folder["id"], "image/jpeg")

        # Resize the image
        resized_image = resize_image(watermarked_image)

        # Get the subfolder "SocMed" if it exists, otherwise create one
        socmed_folder = get_or_create_subfolder(file["parents"][0], "SocMed")

        # Save the resized image to the "SocMed" folder
        save_image_to_drive(resized_image, socmed_folder["id"], "image/jpeg")